In [ ]:
# code for loading in data set from:
# https://pythonbasics.org/how-to-load-machine-learning-data-in-python/
import numpy as np
filename = 'Data/CS170_Small_Data__82.txt'
raw_data = open(filename, 'rt')
data = np.loadtxt(raw_data)
print(data.shape)
print(data)

In [8]:
def LeaveOneOutCrossValid(data, current_set, feature_to_add):
    # function stub for now
    # Leave one out cross validation, returns the accuracy
    # K = num(rows)
    rows = data.shape[0]
    for i in range(rows):
        ObjToClassify = data[i,1:]
        LabelObjToClassify = data[i,0]

        #print(f"Looping over i, at the {i} location")
        #print(f"The {i}'th object is in class {LabelObjToClassify}")
        NearestNeighborDistance = np.inf
        NearestNeighborLocation = np.inf
        for k in range(rows):
            if not k == i: # except self
                distance = np.sqrt(np.sum((ObjToClassify - data[k,1:]) ** 2))
                print(f"Ask if {i} is nearest neighbor with {k}")
                print("Distance: ", distance)
    accuracy = 1
    return accuracy

def feature_search(data):
    CurrentSet = set()

    for i in range(1, data.shape[1]):
        print(f"On the {i}th level of the search tree")
        FeatureToAddAtThisLevel = set()
        BestSoFarAccuracy = 0
        
        for k in range(1, data.shape[1]):
            if not CurrentSet.intersection(set({k})):
                print(f"--Considering adding the {k} feature")
                accuracy = LeaveOneOutCrossValid(data,CurrentSet,k+1)

                if accuracy > BestSoFarAccuracy:
                    BestSoFarAccuracy = accuracy
                    FeatureToAddAtThisLevel = k
        CurrentSet.add(FeatureToAddAtThisLevel) 
        print(f"On level {i} i added feature {FeatureToAddAtThisLevel} to current set")

In [ ]:
#feature_search(data)
LeaveOneOutCrossValid(data,0,0)

In [9]:
filename = 'Data/SampleData/David_Small_Data.txt'
Raw_data = open(filename, 'rt')
Data = np.loadtxt(Raw_data)
#print(Data.shape)
#print(Data)
LeaveOneOutCrossValid(Data,0,0)

Ask if 0 is nearest neighbor with 1
Distance:  3.4673399407157888
Ask if 0 is nearest neighbor with 2
Distance:  2.971072891895854
Ask if 0 is nearest neighbor with 3
Distance:  2.0191034337565212
Ask if 0 is nearest neighbor with 4
Distance:  2.9907792597378187
Ask if 0 is nearest neighbor with 5
Distance:  3.3240213867233397
Ask if 0 is nearest neighbor with 6
Distance:  3.510923744974809
Ask if 0 is nearest neighbor with 7
Distance:  2.9454455589534123
Ask if 0 is nearest neighbor with 8
Distance:  3.6292470762098334
Ask if 0 is nearest neighbor with 9
Distance:  3.3391860597085463
Ask if 1 is nearest neighbor with 0
Distance:  3.4673399407157888
Ask if 1 is nearest neighbor with 2
Distance:  3.7244479860862185
Ask if 1 is nearest neighbor with 3
Distance:  3.9561654686543037
Ask if 1 is nearest neighbor with 4
Distance:  3.5571223695197496
Ask if 1 is nearest neighbor with 5
Distance:  2.4290174640703923
Ask if 1 is nearest neighbor with 6
Distance:  4.025633067830859
Ask if 1 is n

1